In [1]:
import bs4 as bs
from urllib.request import Request, urlopen
import re
import requests
import nltk

import heapq

In [2]:
# text_source = "https://www.sciencedirect.com/science/article/pii/S0004370215000910"
#text_source = "https://www.gutenberg.org/cache/epub/70503/pg70503-images.html"
#request_site = Request(text_source, headers={"User-Agent": "Mozilla/5.0"})
#article = urlopen(request_site).read()
#parsed_article = bs.BeautifulSoup(article, "lxml")
#paragraphs = parsed_article.find_all("p")
article_text = "Wikis are enabled by wiki software, otherwise known as wiki engines. A wiki engine, being a form of a content management system, differs from other web-based systems such as blog software, in that the content is created without any defined owner or leader, and wikis have little inherent structure, allowing structure to emerge according to the needs of the users. Wiki engines usually allow content to be written using a simplified markup language and sometimes edited with the help of a rich-text editor. There are dozens of different wiki engines in use, both standalone and part of other software, such as bug tracking systems. Some wiki engines are free and open-source, whereas others are proprietary. Some permit control over different functions; for example, editing rights may permit changing, adding, or removing material. Others may permit access without enforcing access control. Other rules may be imposed to organize content."

In [3]:
#for p in paragraphs:
#    article_text += p.text
# Remove Square Brackets and Extra Spaces
article_text = re.sub(r"\[[0-9]*\]", " ", article_text)
article_text = re.sub(r"\s+", " ", article_text)

In [4]:
# Remove special characters and digits
formatted_article_text = re.sub("[^a-zA-Z]", " ", article_text)
formatted_article_text = re.sub(r"\s+", " ", formatted_article_text)
#print("\n\n\n The source of original article used is: \n")
#print(text_source)

In [5]:
sentence_list = nltk.sent_tokenize(article_text)
stopwords = nltk.corpus.stopwords.words("english")
word_frequencies = {}
for word in nltk.word_tokenize(formatted_article_text):
    if word not in stopwords:
        if word not in word_frequencies.keys():
            word_frequencies[word] = 1
        else:
            word_frequencies[word] += 1
maximum_frequncy = max(word_frequencies.values())
for word in word_frequencies.keys():
    word_frequencies[word] = word_frequencies[word] / maximum_frequncy

In [6]:
sentence_scores = {}
for sent in sentence_list:
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_frequencies.keys():
            if len(sent.split(" ")) < 30:
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]
                    
summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)
summary = " ".join(summary_sentences)
print("\n\n\nEnglish Language Summary of given text is:\n ")
print(summary)
# print(summary_sentences)
for x in summary_sentences:
    x = re.sub(r"\s+", " ", x)




English Language Summary of given text is:
 
Wiki engines usually allow content to be written using a simplified markup language and sometimes edited with the help of a rich-text editor. There are dozens of different wiki engines in use, both standalone and part of other software, such as bug tracking systems. Wikis are enabled by wiki software, otherwise known as wiki engines. Some permit control over different functions; for example, editing rights may permit changing, adding, or removing material. Others may permit access without enforcing access control. Some wiki engines are free and open-source, whereas others are proprietary. Other rules may be imposed to organize content.


In [7]:
def translate_text(text, source_lang, target_lang):
    translated_text = ""
    length = len(text)
    parts = [text[i:i+500] for i in (0, length, 500)]
    for part in parts:
        url = "https://api.mymemory.translated.net/get"
        params = {
            "q": part,
            "langpair": f"{source_lang}|{target_lang}",
        }
        response = requests.get(url, params=params)
        data = response.json()
        translated_text += data["responseData"]["translatedText"]
    return translated_text


In [8]:
query=summary
translated=translate_text(query,"en","ne")

In [9]:
translated

'Wiki इन्जिनहरूले सामान्यतया सामग्रीलाई सरलीकृत मार्कअप भाषा प्रयोग गरेर लेख्न र कहिलेकाहीँ रिच-टेक्स्ट सम्पादकको मद्दतले सम्पादन गर्न अनुमति दिन्छ। त्यहाँ दर्जनौं बिभिन्न विकी इन्जिनहरू प्रयोगमा छन्, दुबै स्ट्यान्डअलोन र अन्य सफ्टवेयरको अंश, जस्तै बग ट्र्याकिङ प्रणालीहरू। विकीहरू विकी सफ्टवेयरद्वारा सक्षम हुन्छन्, अन्यथा विकी इन्जिनहरू भनिन्छ। केहीले विभिन्न प्रकार्यहरूमा नियन्त्रण गर्न अनुमति दिन्छ; उदाहरणका लागि, सम्पादन अधिकारहरूले सामग्री परिवर्तन गर्न, थप्न वा हटाउने अनुमति दिन सक्छ। अरूले लागू नगरी पहुँच अनुमति दिन सक्छन्NO QUERY SPECIFIED. EXAMPLE REQUEST: GET?Q=HELLO&LANGPAIR=EN|ITपहुँच नियन्त्रण। केही विकी इन्जिनहरू नि:शुल्क र खुला स्रोत हुन्, जबकि अरूहरू स्वामित्वमा छन्। सामग्री व्यवस्थित गर्न अन्य नियमहरू लागू गर्न सकिन्छ।'